In [2]:
import pandas as pd
import numpy as np
import json

In [ ]:
full_conversation_path = '/playpen-ssd/smerrill/llm_decisions/results_simulation/3.1/T0.4_P0.6_K20_R1.3/full_conversation.json's

In [ ]:
with open(full_conversation_path, 'r') as f:
    full_conversation = json.load(f)

print(f"Full conversation entries: {len(full_conversation)}")

Full conversation entries: 27
Public votes entries: 7
Private votes entries: 7


In [ ]:
import json

def detect_directed_question(agent, speaker, content, participants):
    """
    Detect if an utterance is a question and who it is directed to.
    
    Returns:
        dict with keys: "is_question" (bool), "directed_to" (str or None)
    """
    participants_str = ", ".join(participants)
    
    prompt = f"""
You are analyzing a school board meeting transcript.

Participants: {participants_str}

Task:
1. Determine if the speaker is asking a question.
2. If yes, detect the intended recipient from the participants list.
3. Only output in JSON format:
{{"is_question": true/false, "directed_to": "<participant_name or null>"}}

Constraints:
- Only use the participants listed above.
- If the question is general or rhetorical, set "directed_to": null.
- Only output JSON.

Utterance:
Speaker: {speaker}
Content: {content}

Output:
"""
    response = agent.generate_response(prompt)
    
    # Parse JSON safely
    try:
        result = json.loads(response)
        # normalize null to Python None
        if result.get("directed_to") == "null":
            result["directed_to"] = None
    except json.JSONDecodeError:
        result = {"is_question": False, "directed_to": None}
    
    return result


Private Votes: {'yes': 6, 'no': 0, 'abstain': 1}
Public Votes: {'yes': 5, 'no': 0, 'abstain': 2}


In [ ]:
def analyze_transcript(agent, transcript, participants):
    """
    Analyze a transcript and detect directed questions.
    
    Returns a list of dicts per utterance:
    {
        "speaker": ...,
        "content": ...,
        "is_question": True/False,
        "directed_to": participant or None
    }
    """
    results = []
    for turn in transcript:
        speaker = turn['speaker']
        content = turn['content']
        detection = detect_directed_question(agent, speaker, content, participants)
        results.append({
            "speaker": speaker,
            "content": content,
            **detection
        })
    return results


In [ ]:
participants = ["ellenosborne", "judyle", "grahampaige", "katrinacallsen",
                "kateacuff", "davidoberg", "jonnoalcaro"]

# Suppose `agent` is your LLaMA70B agent
results = analyze_transcript(agent, transcript, participants)

# Example output:
for r in results:
    print(r)
